# Numerai

In [5]:
# Initialize NumerAPI - the official Python API client for Numerai
from numerapi import NumerAPI
napi = NumerAPI()

# Print all files available for download in the latest v4.1 dataset
[f for f in napi.list_datasets() if f.startswith("v4.1")]

2023-08-22 19:21:36,415 ERROR numerapi.utils: Error Connecting: HTTPSConnectionPool(host='api-tournament.numer.ai', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1002)')))


KeyError: 'data'

In [4]:
import pandas as pd
import json

# Download the training data and feature metadata
# This may take a few minutes 🍵
napi.download_dataset("v4.1/train.parquet");
napi.download_dataset("v4.1/features.json");

# Load the training data but only the "small" subset of features to save time and memory
# In practice you will want to use all the features to maximize your model's performance
feature_metadata = json.load(open("v4.1/features.json")) 
feature_cols = feature_metadata["feature_sets"]["small"]
training_data = pd.read_parquet("v4.1/train.parquet", columns= ["era"] + feature_cols + ["target"]) 

# Print the training data
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 5)
training_data

2023-08-22 19:21:26,205 ERROR numerapi.utils: Error Connecting: HTTPSConnectionPool(host='api-tournament.numer.ai', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1002)')))


KeyError: 'data'